In [3]:
import json
import os
import tempfile
from pathlib import Path
import boto3
from botocore.exceptions import NoCredentialsError, ClientError
import logging
import cv2
import subprocess
import re

from improved_aws_data_loader import AWSVideoFrameExtractor

In [4]:
#!/usr/bin/env python3

"""
개선된 AWS Data Loader로 새로운 multitask_test_dataset.jsonl 처리 예시
"""

from improved_aws_data_loader import AWSVideoFrameExtractor
import logging

# 로깅 레벨 설정 (원하는 경우)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# AWS 자격증명 설정 (환경변수 사용 권장)
extractor = AWSVideoFrameExtractor(
    region_name='ap-northeast-2'
)

# JSONL 파일 처리
jsonl_file_path = "/home/compu/test_suchae/EAGLE/test_suchae/multitask_test_dataset.jsonl"

try:
    # 먼저 데이터셋 구조 분석 (선택사항)
    print("=== 데이터셋 구조 분석 ===")
    task_counts, sample_data = extractor.analyze_dataset_structure(jsonl_file_path, max_lines=20)
    
    print("\n=== 프레임 추출 시작 ===")
    
    # 옵션 1: 기존 파일들을 모두 삭제하고 새로 시작
    # processed_data = extractor.process_jsonl_file(jsonl_file_path, use_ffmpeg=True, clear_existing=True)
    
    # 옵션 2: 기존 파일은 건너뛰고 없는 것만 추출 (기본값, 권장)
    processed_data = extractor.process_jsonl_file(jsonl_file_path, use_ffmpeg=True, clear_existing=False)
    
    # 처리된 데이터 사용 예시
    print(f"\n=== 처리 결과 요약 ===")
    print(f"총 처리된 샘플: {len(processed_data)}개")
    
    task_summary = {}
    for data in processed_data:
        task = data.get('task', 'unknown')
        if task not in task_summary:
            task_summary[task] = {'samples': 0, 'images': 0}
        task_summary[task]['samples'] += 1
        task_summary[task]['images'] += len(data.get('image_info', []))
    
    for task, info in task_summary.items():
        print(f"{task}: {info['samples']}개 샘플, {info['images']}개 이미지")
    
    # 처리된 데이터 상세 출력 (처음 3개만)
    print(f"\n=== 처리 결과 상세 (처음 3개 샘플) ===")
    for i, data in enumerate(processed_data[:3]):
        print(f"\n[{i+1}] Sample ID: {data['sample_id']}")
        print(f"    Task: {data.get('task', 'unknown')}")
        
        for img_info in data['image_info']:
            scenario = img_info.get('scenario', 'default')
            print(f"  Image {img_info['image_id']} ({scenario}): {img_info['path']}")
            print(f"    View: {img_info['view']}")
            print(f"    추출 시간: {img_info['extracted_timestamp']}초")
            print(f"    비디오 비율: {img_info['video_ratio']}")
            if 'source_video' in img_info:
                print(f"    소스 비디오: {img_info['source_video']}")
            
except Exception as e:
    logger.error(f"메인 처리 오류: {str(e)}")

INFO:improved_aws_data_loader:AWS S3 클라이언트가 성공적으로 초기화되었습니다.
INFO:improved_aws_data_loader:데이터셋 구조 분석 중: /home/compu/test_suchae/EAGLE/test_suchae/multitask_test_dataset.jsonl
INFO:improved_aws_data_loader:=== 데이터셋 구조 분석 결과 ===
INFO:improved_aws_data_loader:Task: task3
INFO:improved_aws_data_loader:  - 샘플 수: 9
INFO:improved_aws_data_loader:  - S3 정보 키: ['bucket', 'base_path', 'views']
INFO:improved_aws_data_loader:  - 이미지 수: 3
INFO:improved_aws_data_loader:  - 시나리오 정보: False
INFO:improved_aws_data_loader:
INFO:improved_aws_data_loader:Task: task1
INFO:improved_aws_data_loader:  - 샘플 수: 3
INFO:improved_aws_data_loader:  - S3 정보 키: ['bucket', 'base_path', 'views']
INFO:improved_aws_data_loader:  - 이미지 수: 4
INFO:improved_aws_data_loader:  - 시나리오 정보: False
INFO:improved_aws_data_loader:
INFO:improved_aws_data_loader:Task: task2
INFO:improved_aws_data_loader:  - 샘플 수: 8
INFO:improved_aws_data_loader:  - S3 정보 키: ['bucket', 'base_path', 'second_base_path', 'scenario1_views', 'scenario2_views'

=== 데이터셋 구조 분석 ===

=== 프레임 추출 시작 ===


INFO:improved_aws_data_loader:비디오 다운로드 완료: external_dataset/v1.0/aloha_mobile/episode_000022/fix_center.mp4 -> /tmp/temp_videos_fle71_kj/default_fix_center.mp4
INFO:improved_aws_data_loader:비디오 다운로드 완료: default_fix_center
INFO:improved_aws_data_loader:이미지가 이미 존재함, 건너뜀: /tmp/aws_video_processor_vp8kaw25/fix_center_episode_000022_20250831_135632_7d328647_0.50_20.0s.jpg
INFO:improved_aws_data_loader:프레임 추출 성공: default - fix_center - 20.0초
INFO:improved_aws_data_loader:비디오 다운로드 완료: external_dataset/v1.0/aloha_mobile/episode_000022/ego_right.mp4 -> /tmp/temp_videos_fle71_kj/default_ego_right.mp4
INFO:improved_aws_data_loader:비디오 다운로드 완료: default_ego_right
INFO:improved_aws_data_loader:이미지가 이미 존재함, 건너뜀: /tmp/aws_video_processor_vp8kaw25/ego_right_episode_000022_20250831_135633_7d328647_0.50_20.0s.jpg
INFO:improved_aws_data_loader:프레임 추출 성공: default - ego_right - 20.0초
INFO:improved_aws_data_loader:비디오 다운로드 완료: external_dataset/v1.0/aloha_mobile/episode_000022/ego_left.mp4 -> /tmp/temp_videos

KeyboardInterrupt: 